In [18]:
import camelot
import numpy as np
import pandas as pd
import json
import os

In [19]:
pd_df_str,concatenated_df_lst,concat_string_final,df_str,concat_df_final = '','','','',''
df = []

with open('config.json') as f:
    config = json.load(f)

in_file = config['pdf_input_location']
password = config['password']
file_list = [x.split('.')[0] for x in os.listdir(in_file['pdf_input_location'])]
out_file = config['csv_output_location']

In [20]:
def hdfc_yrly_stmt_pdf_to_csv(pdf_path,out_path,password):
    
    global pd_df_str,tables,pdf_df

    #print('Reading tables')
    tables = camelot.read_pdf(pdf_path,pages='all',password=password)
    
    #print('Creating pd_df_str')
    for i in range(len(tables)):
        pd_df_str += """tables[{0}].df,""".format(i)

    #print("Creating concat_string")
    concat_string = """pdf_df = pd.concat(["""+pd_df_str[:-1]+"""],ignore_index=True,axis = 0)"""

    #print("Executing concat_string")
    exec(concat_string,globals())

    #print("pdf_df and pdf_df2 created")
    pdf_df2 = pdf_df    

    new_header = pdf_df2.iloc[0]
    #print(new_header)
    pdf_df3 = pdf_df2[1:]
    pdf_df3.columns = new_header
    pdf_df3 = pdf_df3.reset_index(drop=True)
    pdf_df3['Chq. / Ref No.'] = np.where(((pdf_df3['Chq. / Ref No.']=='') & (pdf_df3['Date']!='')),pdf_df3['Narration'],pdf_df3['Chq. / Ref No.'])
    all_col_list = pdf_df3.iloc[0].to_list()    
    pdf_df3.to_csv(out_path,header=True,index=False)
    

    new_header,concat_string,pd_df_str = '','',''
    del tables
    del pdf_df,pdf_df2
    return pdf_df3

In [21]:
for i in range(len(file_list)):
    df.append(str("df{0}".format(i)))
    df_str += """df{0},""".format(i)

concat_df_final = """concatenated_df = pd.concat(["""+df_str[:-1]+"""],ignore_index=True,axis = 0)"""

In [22]:
for i in range(len(file_list)):
    print(' ',i)
    print('**In-put file {0} read and conversion to csv started**'.format(in_file['pdf_input_location']+file_list[i]+'.pdf'))
    df[i] = pd.DataFrame(hdfc_yrly_stmt_pdf_to_csv(in_file['pdf_input_location']+file_list[i]+'.pdf',out_file['csv_output_location']+file_list[i]+'.csv',password['password']))
    print('**Output file {0} created**'.format(out_file['csv_output_location']+file_list[i]+'.csv'))
    print(' ')

  0
**In-put file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\yearly statment\input\0968XXXXXX7192_0648a7de_01Jan2022_TO_01Dec2022_205959177.pdf read and conversion to csv started**
**Output file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\yearly statment\output\0968XXXXXX7192_0648a7de_01Jan2022_TO_01Dec2022_205959177.csv created**
 
  1
**In-put file C:\Users\pc\Desktop\Dheeraj\Finance analysis\pdf\yearly statment\input\0968XXXXXX7192_41589d1c_01Jan2023_TO_01Dec2023_205431694.pdf read and conversion to csv started**


In [ ]:
exec(concat_df_final)